In [1]:
from __future__ import division, print_function
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
import string
import gensim
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
from datetime import datetime
from ast import literal_eval

Using TensorFlow backend.
/Users/thomasvanorden/anaconda3/envs/NTMI/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/thomasvanorden/anaconda3/envs/NTMI/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/thomasvanorden/anaconda3/envs/NTMI/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])

In [2]:
def extend_dataframe(filename):
    data = pd.read_csv(filename)
    data.tokens = data.tokens.apply(literal_eval)
    data.lemmed = data.lemmed.apply(literal_eval)
    data.stemmed = data.stemmed.apply(literal_eval)
    
    data["label"] = data["label"].astype(bool)
    data["pos"] = data["label"]
    data["neg"] = ~data["label"]
    
    return data


def create_vocab(data):
    all_words = [word for tokens in data for word in tokens]
    sentence_lengths = [len(tokens) for tokens in data]

    VOCAB = list(set(all_words))
    
    return VOCAB


def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)

    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]

    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, len(vectorized))

    return averaged


def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = clean_comments['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)


def get_tokenizer_sequences(data, VOCAB):
    tokenizer = Tokenizer(num_words=len(VOCAB), lower=True, char_level=False)
    tokenizer.fit_on_texts(data.tolist())

    sequences = tokenizer.texts_to_sequences(data.tolist())
    
    return sequences, tokenizer


def preprocess_cleaned_data(data, VOCAB):
    training_sequences, tokenizer = get_tokenizer_sequences(data, VOCAB)
    data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)
    print('Found %s unique tokens.' % len(tokenizer.word_index))

    return tokenizer, data


def get_train_embedding_weights(tokenizer, model, EMBEDDING_DIM):
    weights = np.zeros((len(tokenizer.word_index)+1, EMBEDDING_DIM))

    for word,index in tokenizer.word_index.items():
        weights[index,:] = model.wv[word] if word in model.wv else np.random.rand(EMBEDDING_DIM)
        
    return weights

In [19]:
#------------HYPERPARAMETERS-----------#
# Parameter for input data
INPUT = 'lemmed' # 'lemmed' 'stemmed' 'tokens'

# Parameters for embedding the sequences
MIN_COUNT = 2
EMED = 'skipgram' # 'skipgram'
WINDOW_SIZE = 10
ITER = 10

# Parameters for creating the model
MAX_SEQUENCE_LENGTH = 500
# EMBEDDING_DIM = None
EMED_TRAINABLE = True # Train embedding weights


# Parameters for fitting the CNN model
NUM_EPOCHS = 8
BATCH_SIZE = 64

#--------------------------------------#

In [4]:
data_training = extend_dataframe("cleaned_data/cleaned_data_train_with_lem_stem.csv")
data_test = extend_dataframe("cleaned_data/cleaned_data_test_with_lem_stem.csv")

In [5]:
VOCAB_training = create_vocab(data_training[INPUT])

In [6]:
from collections import Counter

training_sents = data_training[INPUT]
test_sents = data_test[INPUT]

training_lengths =[len(sentence) for sentence in training_sents]
test_lengths =[len(sentence) for sentence in test_sents]

print("Average sentence length training:", Counter(training_lengths).most_common(1)[0][0])
print("Average sentence length test:", Counter(test_lengths).most_common(1)[0][0])

Average sentence length training: 128
Average sentence length test: 127


In [7]:
# fig = plt.figure(figsize = (6,4))

# plt.hist(training_lengths, bins=range(min(training_lengths), max(training_lengths) + 1, 1), 
#               alpha=0.5, color="red", label="Training")
# plt.hist(test_lengths, bins=range(min(test_lengths), max(test_lengths) + 1, 1),
#               alpha=0.5, color="blue", label="Test")

# plt.legend()
# plt.xlabel("Sentence length")
# plt.ylabel("Counts")

# fig.savefig('sentence_lengths.png', bbox_inches = "tight", dpi=300)

In [8]:
# from nltk.probability import FreqDist
import matplotlib.pyplot as plt


# def create_corpus(data):
#     corpus_training = []
#     for sent in data:
#         for word in sent:
#             corpus_training.append(word)
    
#     return corpus_training

# corpus_training = create_corpus(data_training[INPUT])
# corpus_test = create_corpus(data_test[INPUT])

# # Tokens
# # Total words in corpus: 5823763
# # Unique words in corpus: 121230

# # Lemmed
# # Total words in corpus: 5823763
# # Unique words in corpus: 110541

# # Stemmed
# # Total words in corpus: 5823763
# # Unique words in corpus: 92824

In [9]:
# fig = plt.figure(figsize = (6,4))

# # set width of bar
# barWidth = 0.25
 
# # set height of bar
# training = [len((corpus_training))]
# test = [len((corpus_test))]
 
# # Set position of bar on X axis
# r1 = np.arange(len(training))
# r2 = [x + barWidth for x in r1]
 
# # Make the plot
# plt.bar(r1, training, color="red", alpha=0.4, width=barWidth, edgecolor='white', label='Training')
# plt.bar(r2, test, color="blue", alpha=0.4, width=barWidth, edgecolor='white', label='Test')
 
# # Add xticks on the middle of the group bars
# plt.ylabel("Counts")
# plt.xticks([0.125], ['Total words'])
 
# # Create legend & Show graphic
# plt.legend()
# plt.show()

# fig.savefig('total_words.png', bbox_inches = "tight", dpi=300)

In [10]:
# print("Total words in corpus:",len(corpus_training))
# print("Unique words in corpus:", len(set(corpus_training)))

# fig = plt.figure(figsize = (10,4))

# plt.gcf().subplots_adjust(bottom=0.15) # to avoid x-ticks cut-off
# fdist = FreqDist(corpus_training)

# fdist.plot(30)
# plt.show()
# fig.savefig('orignal_training.png', bbox_inches = "tight", dpi=300)

In [11]:
# Create and fit tokenizer on training data
tokenizer = Tokenizer(num_words=len(VOCAB_training), lower=True, char_level=False)
tokenizer.fit_on_texts(data_training[INPUT])

In [12]:
# Preprocess training data
training_sequences = tokenizer.texts_to_sequences(data_training[INPUT])
train_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [13]:
# Preprocess test data
test_sequences = tokenizer.texts_to_sequences(data_test[INPUT])
test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [14]:
# Get the current date and time, for saving convience
dt_string = datetime.now().strftime("%d-%m-%Y_%H:%M:%S")

In [15]:
# Save the test data for further use
!mkdir -p saved_test_data
np.save("saved_test_data/test_data_"+dt_string, test_data)

In [20]:
if EMED == 'cbow':
    # Create Cbow model
    model = gensim.models.Word2Vec([tokens for tokens in data_training[INPUT]], min_count = MIN_COUNT,
                                    size = MAX_SEQUENCE_LENGTH, window = WINDOW_SIZE, sg=0, workers=6, iter=ITER)
else:
    # Create Skip Gram model
    model = gensim.models.Word2Vec([tokens for tokens in data_training[INPUT]], min_count = MIN_COUNT, 
                                        size = MAX_SEQUENCE_LENGTH, window = WINDOW_SIZE, sg = 1, workers=6, iter=ITER)

In [21]:
# Calculate embedding weights for the model
train_embedding_weights = get_train_embedding_weights(tokenizer, model, MAX_SEQUENCE_LENGTH)

# CNN

https://github.com/saadarshad102/Sentiment-Analysis-CNN/blob/master/Notebook.ipynb

In [45]:
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index, emb_trainable=False):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=emb_trainable)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    convs = []
    filter_sizes = [2,3,4,5,6]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=200, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = GlobalMaxPooling1D()(l_conv)
        convs.append(l_pool)


    l_merge = concatenate(convs, axis=1)

    x = Dropout(0.1)(l_merge)  
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    
    return model

In [46]:
label_names = ['pos', 'neg']

In [47]:
model = ConvNet(train_embedding_weights, MAX_SEQUENCE_LENGTH, train_embedding_weights.shape[0], MAX_SEQUENCE_LENGTH, 
                len(list(label_names)), emb_trainable=EMED_TRAINABLE)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 500)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 500, 100)     11054200    input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 499, 200)     40200       embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, 498, 200)     60200       embedding_2[0][0]                
____________________________________________________________________________________________

In [48]:
# Plot the model
# import keras
# import pydotplus
# from keras.utils.vis_utils import model_to_dot
# keras.utils.vis_utils.pydot = pydot
# from keras.utils import plot_model
# plot_model(model, to_file='model.png', dpi=100, show_shapes=True, expand_nested=True)

In [49]:
hist = model.fit(train_data, data_training[label_names].values, epochs=NUM_EPOCHS, validation_split=0.1, shuffle=True, batch_size=BATCH_SIZE, use_multiprocessing=True)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 22500 samples, validate on 2500 samples
Epoch 1/8
22500/22500 [==============================] - 490s 22ms/step - loss: 0.4181 - acc: 0.8003 - val_loss: 0.4377 - val_acc: 0.8038
Epoch 2/8
22500/22500 [==============================] - 2491s 111ms/step - loss: 0.1992 - acc: 0.9259 - val_loss: 0.3369 - val_acc: 0.8658
Epoch 3/8
22500/22500 [==============================] - 515s 23ms/step - loss: 0.0825 - acc: 0.9742 - val_loss: 0.1666 - val_acc: 0.9366
Epoch 4/8
22500/22500 [==============================] - 4877s 217ms/step - loss: 0.0216 - acc: 0.9941 - val_loss: 0.7218 - val_acc: 0.8154
Epoch 5/8
22500/22500 [==============================] - 710s 32ms/step - loss: 0.0045 - acc: 0.9994 - val_loss: 0.3402 - val_acc: 0.9114
Epoch 6/8
22500/22500 [==============================] - 1070s 48ms/step - loss: 0.0012 - acc: 1.0000 - val_loss: 0.6081 - val_acc:

KeyboardInterrupt: 

In [ ]:
!mkdir -p saved_model

name = f"MODEL_{EMED}_{INPUT}_" + dt_string
dir_ = f'saved_model/{name}'

print(f"Saving model as {name} in saved_model/...")
model.save(dir_)
print("DONE")

In [ ]:
# Plot training & validation accuracy values
plt.plot(hist.hist['acc'])
plt.plot(hist.hist['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(hist.hist['loss'])
plt.plot(hist.hist['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()